# 2D Hyperparameter Tuning for Fuzzy C-Means

Description

In [1]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import os
import sys

from pathlib import Path

path = (
    Path
    .cwd()
    .parent
    .parent
    .joinpath('warbler.py')
)

os.chdir(path)
sys.path.append(path)

In [11]:
import numpy as np

from constant import SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.scorer import (
    CalinskiHarabaszScore,
    DaviesBouldinIndex,
    FukuyamaSugenoIndex,
    PartitionCoefficient,
    PartitionEntropyCoefficient,
    Scorer,
    SilhouetteScore,
    SumOfSquaredErrors,
    XieBeniIndex
)
from datatype.search import GridSearch
from datatype.settings import Settings
from datatype.voronoi import Builder, VoronoiFCM
from fcmeans import FCM

In [3]:
dataset = Dataset('segment')
dataframe = dataset.load()

In [4]:
x = np.concatenate((
    [dataframe.umap_x_2d],
    [dataframe.umap_y_2d]
))

x = x.transpose()

In [ ]:
grid = {
    'm': np.arange(1.5, 2.5, 0.1),
    'max_iter': np.arange(50, 200, 50),
    'n_clusters': np.arange(2, 20)
}

strategies = [
    CalinskiHarabaszScore(),
    DaviesBouldinIndex(),
    FukuyamaSugenoIndex(),
    PartitionCoefficient(),
    PartitionEntropyCoefficient(),
    SilhouetteScore(),
    SumOfSquaredErrors(),
    XieBeniIndex()
]

In [ ]:
for strategy in strategies:
    name = str(strategy)
    print(name)

    scorer = Scorer()
    scorer.strategy = strategy

    search = GridSearch()
    search.grid = grid
    search.scorer = scorer

    data = deepcopy(x)
    search.fit(data)

    filename = repr(scorer.strategy)
    search.export(filename)

    fcm = FCM(**search.best_parameters)
    fcm.fit(data)

    label = fcm.predict(data)
    dataframe['fcm_label_2d'] = label

    # Load default settings
    path = SETTINGS.joinpath('voronoi.json')
    settings = Settings.from_file(path)
    settings.name = str(strategy)

    unique = dataframe.fcm_label_2d.unique()

    by = ['duration']

    ascending = [False]

    dataframe = dataframe.sort_values(
        ascending=ascending,
        by=by
    )

    coordinates = [
        dataframe.umap_x_2d,
        dataframe.umap_y_2d
    ]

    embedding = np.column_stack(coordinates)

    spectrogram = dataframe.original_array.to_numpy()
    label = dataframe.fcm_label_2d.to_numpy()

    voronoi = VoronoiFCM()

    voronoi.builder = Builder()
    voronoi.embedding = embedding
    voronoi.label = label
    voronoi.spectrogram = ~spectrogram
    voronoi.settings = settings
    voronoi.unique = unique

    component = voronoi.build()

    figure = component.get('figure')

    filename = filename + '.png'

    voronoi.save(
        figure=figure,
        filename=filename
    )